In [1]:
import sys
sys.path.append('../')

from database import *
import database.models as models

# load full text from db
aps_details_single = models.Review().query.filter(models.Review.status.in_(('needs_crosscheck', 'done'))).filter(models.Review.review_type == 'single_focus').all()
aps_details_w_title = [i for i in aps_details_single if i.reviewed_book_title != '' and i.reviewed_book_title is not None]
publishers = [i.reviewed_book_publisher for i in aps_details_single if i.reviewed_book_publisher !='']
known_publishers = list(set(publishers))
len(aps_details_w_title)

558

In [2]:
from application.review import ReviewObject
from application.name_obj_classes import remove_punct

In [4]:
review_instances = []
for r in aps_details_single:
    one_review = ReviewObject(title=r.reviewed_book_title, known_publishers=known_publishers, **r.__dict__)
    review_instances.append(one_review)
review_instances[0].top_publishers

Counter({'american commonwealths boston itoughto. sftfltn co.': 1.0})

In [7]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from fuzzysearch import find_near_matches
from collections import Counter
import re
import string
from nltk.metrics import edit_distance

In [5]:
ground_truth_lists = []
for e,i in enumerate(aps_details_single):
    this_record = {}
    for z in ["reviewed_book_title", "reviewed_author_name", "reviewed_book_publisher", \
              "reviewed_author_viaf_match", "reviewed_book_publisher_viaf_match", "year"]:
        this_record[z] = getattr(i, z)
        
        if z in ["reviewed_book_title", "reviewed_author_name", "reviewed_book_publisher"]:
            tokens = word_tokenize(this_record[z].lower())
            tokens = [j.replace("&", "and") for j in tokens]
            tokens = [j for j in tokens if j not in string.punctuation]
            this_record[z+"_tokens"] = tokens
    this_record["record_tokens"] = this_record["reviewed_book_title_tokens"] + this_record["reviewed_author_name_tokens"] + this_record["reviewed_book_publisher_tokens"] 
    ground_truth_lists.append(this_record)

In [20]:
ground_truth_lists[0]['record_tokens']


['missouri', 'lucien', 'carr', 'houghton', 'mifflin', 'and', 'company']

In [49]:
def compare_candidates_and_labels(c, l):
    """ compares and aligns two token lists """
    # check for whole object and sub-token matches for equal number of tokens... such as surname and publisher name (distance of best alignment)
    if len(c) > len(l):
        longer = c
        shorter = l
    else: 
        longer = l
        shorter = c
    best_distance = sum([len(i) for i in longer])
    best_start = 0
    
    for en, comp in enumerate(longer):
        summed_distances = 0
        for e, token in enumerate(shorter):
            pos = en+e
            if pos < len(longer):
                s = edit_distance(token, longer[pos])
            else:
                s = len(token)
            summed_distances += s
        print(comp, shorter, summed_distances)
        if summed_distances < best_distance:
            best_distance = summed_distances
            best_start = en
    return (best_distance, shorter, longer[best_start: best_start+len(shorter)])

In [42]:
def get_most_common(counter):
    try:
        entity = counter.most_common(1)[0][0].lower()
    except:
        entity = ""
    return entity

for e,i in enumerate(review_instances):
    title = get_most_common(i.top_titles)
    pub = get_most_common(i.top_publishers)
    auth = get_most_common(i.top_authors)
    
    for key, comp in [("reviewed_book_title_tokens", title), ("reviewed_author_name_tokens", auth), ("reviewed_book_publisher_tokens", pub)]:
        print(ground_truth_lists[e][key][0], comp)
        #print(edit_distance(ground_truth_lists[e][key], comp))
    print('\n')

    # check if candidate is an empty string
    # replace things like ampersands, possessives, punctuation
    # tokenize and limit title to four tokens ... what if the title candidate is words 4,5,6 and 7? 
    # remove function words
    # standardize initial tokens
    # handle more than one author in ground truth

missouri missouri a bone of contention
lucien carr
houghton american commonwealths boston itoughto. sftfltn co.


the lord byron self-revealed
lord johnson
john scribner's


carnival carnival
compton aldavini
d. d. appleton & company


photographic 
george fox
e. e. b. treat


life life and letters of dante gabriel rossetti
william william michael rossetti


IndexError: list index out of range

In [48]:


#compare_candidates_and_labels(['bone', 'with', 'contention'], ['missouri', 'a', 'bone', 'of', 'contention'])           
compare_candidates_and_labels(['houghton'], ['american', 'commonwealths', 'boston', 'itoughto', 'sftfltn' 'co'])
            
    # handle alignment tails by adding tail token lengths to edit distance 

# gather best distances 
# narrow the match list by best fields and year
# pick top from what remains ... rank by total distance, subsort by number of perfect matches?

american ['houghton'] 7
commonwealths ['houghton'] 11
boston ['houghton'] 4
itoughto ['houghton'] 3
sftfltnco ['houghton'] 8


(3, ['houghton'], ['itoughto'])

In [ ]:
all_scores = []
all_text_merged = []

# loop candidate data
for e,t in enumerate(review_objects):
    # true label is titles[e]
    # get text from various attributes (title, publisher, author) 
    # tokenize
    # remove function words and punctuation? 
    text_merged = [] 
    for z in [t.title_candidates, t.author_candidates, t.publisher_candidates]:
        this_list = []
        #z is a dictionary or Counter
        for k,v in z.items():
            #screen out 'review no title'
            tokens = word_tokenize(k.lower())
            tokens = [j.replace("&", "and") for j in tokens]
            tokens = [j for j in tokens if j not in string.punctuation]
            #for r in range(v):
            this_list.extend(tokens)
        match_indices = []
        for n,t in enumerate(this_list):
            if t == 'review':
                try:
                    if this_list[n+1] == 'no':

                        try:
                            if this_list[n+2] == 'title':
                                match_index = n
                                match_indices.append(n)
                        except:
                            pass
                except:
                    pass
        for m in match_indices:
            this_list[m] = ''
            this_list[m+1] = ''
            this_list[m+2] = ''
        this_list = [j for j in this_list if j !='']
        text_merged.extend(this_list)
    all_text_merged.append(text_merged)
    # the true vector becomes comparison_set[-1]
    comparison_set = [Counter(u) for u in ground_truth_lists] + [Counter(text_merged)]
    
    # compare 
    vectorizer = DictVectorizer()
    X = vectorizer.fit_transform(comparison_set)
    vectors = X.toarray()
    # loop all, get similarity, last one is always 1.0
    scores = []
    for v in vectors:
        score = cosine_similarity([v], [vectors[-1]])
        scores.append(score)
    all_scores.append(scores)

In [ ]:
all_text_merged[2]

In [ ]:
# how often is the right answer the top answer? how often is the correct match in the top 5?
output = []
for e, score_grid in enumerate(all_scores):
    df = pd.DataFrame()
    df['score'] = [i[0][0] for i in score_grid][:-1]
    df['title'] = titles
    match = []
    for i in range(len(titles)):
        if e == i:
            match.append('yes')
        else:
            match.append('no')
    df['match'] = match
    output.append(df.sort_values(by="score", ascending=False).reset_index(drop=False))

In [ ]:
top_n = []
accuracy = []

for r in range(1, 562):
    results = []
    for i in output:
        df_n = i.iloc[:r]
        # check if 'yes' is in df_n
        result = len(df_n.loc[df_n['match'] == 'yes']) > 0
        results.append(result)
    acc = len([i for i in results if i])/len(results) 
    top_n.append(r)
    accuracy.append(acc)

In [ ]:
%matplotlib inline
import seaborn as sns
sns.lineplot(x=top_n, y=accuracy)

In [ ]:
top_match = []
top_five = []
top_ten = []
top_25 = []
top_50 = []
top_100 = []
top_200 = []
for i in output:
    df_five = i.iloc[:5]
    df_ten = i.iloc[:10]
    df_25 = i.iloc[:25]
    df_50 = i.iloc[:50]
    df_100 = i.iloc[:100]
    df_200 =i.iloc[:200]
    top_match.append(i.iloc[0]['match'] == 'yes')
    top_five.append(len(df_five.loc[df_five['match'] == 'yes']) > 0)
    top_ten.append(len(df_ten.loc[df_ten['match'] == 'yes']) > 0)
    top_25.append(len(df_25.loc[df_25['match'] == 'yes']) > 0)
    top_50.append(len(df_50.loc[df_50['match'] == 'yes']) > 0)
    top_100.append(len(df_100.loc[df_100['match'] == 'yes']) > 0)
    top_200.append(len(df_200.loc[df_200['match'] == 'yes']) > 0)

In [ ]:
# 55.97% whole vector
len([i for i in top_match if i])/len(top_match)

In [ ]:
# 73% in top five
len([i for i in top_five if i])/len(top_five)

In [ ]:
# 80% in top ten
len([i for i in top_ten if i])/len(top_ten)

In [ ]:
#87.5% in top 25
len([i for i in top_25 if i])/len(top_25)

In [ ]:
#91% in top 50
len([i for i in top_50 if i])/len(top_50)

In [ ]:
len([i for i in top_200 if i])/len(top_200)

In [ ]:
mismatches = []
mismatch_candidate_tokens = []
for e, df in enumerate(output):
    if df.iloc[0]['match'] != 'yes':
        mismatches.append(df)
        mismatch_candidate_tokens.append(all_text_merged[e])
mismatches[2].iloc[0]['title'], mismatches[2].loc[mismatches[2]['match'] == 'yes'].iloc[0]['title'], mismatch_candidate_tokens[2]
mismatches[2].iloc[24]['title']

In [ ]:
import requests
from bs4 import BeautifulSoup
#r = requests.get("https://www.worldcat.org/search?q=ti%3Ahuckleberry+finn&fq=yr%3A1865..1925+%3E+%3E+x0%3Abook&qt=advanced&dblist=638")
# page 2 ... "https://www.worldcat.org/search?q=ti%3Ahuckleberry+finn&fq=yr%3A1865..1925+%3E+%3E+x0%3Abook&dblist=638&start=11&qt=page_number_link"
bs = BeautifulSoup(r.text)
bs.find_all("span", {"class":"itemPublisher"})